In [2]:
import pandas as pd
blast_column_names = ["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore"]                         
gff_file = '/bioinfo/palm/ref/dura/dura_ref.gff'
gff_column_names = ['Seqid', 'Source', 'Type', 'Start', 'End', 'Score', 'Strand', 'Phase', 'Attributes']
bs = 0 #bitscore filtering
n = 3 #hits per gene
flanking = 2000 #flanking regions upstream and downstream
fileA = 'ABA_dura_blastp.tbl'
fileB = 'dura_ABA_blastp.tbl'

# Read data from file A
fileA_data = pd.read_csv(fileA, sep='\t', names=blast_column_names)

# Read data from file B
fileB_data = pd.read_csv(fileB, sep='\t', names=blast_column_names)

# Initialize an empty list to store the results
result_pairs = {}

# Loop through unique values in 'sseqid' column in file A

for sseqid_a in fileA_data['sseqid'].unique():
    count = 0
    # Filter rows in file A where 'sseqid' is equal to the current value in the loop
    filtered_rows = fileA_data[fileA_data['sseqid'] == sseqid_a]
    # Sort the filtered rows by 'bitscore' in descending order
    sorted_rows = filtered_rows.sort_values(by='bitscore', ascending=False)
    # Iterate through the sorted rows
    count = 0
    for _, max_bitscore_row in sorted_rows.iterrows():
        if count < n:
            # Get the value for 'qseqid' in the current row
            qseqid_b = max_bitscore_row['qseqid']

            # Check if the row exists in file B with 'sseqid' == 'b' and 'qseqid' == 'a'
            matching_row = fileB_data[(fileB_data['sseqid'] == qseqid_b) & (fileB_data['qseqid'] == sseqid_a)]

            # If the matching row exists and 'bitscore' is greater than the threshold, add 'a' and 'b' to the result_pairs dictionary
            if not matching_row.empty and max_bitscore_row['bitscore'] > bs:
                sseqid_a = sseqid_a.split('-mRNA')[0]
                qseqid_b = qseqid_b.split('-mRNA')[0]
                if sseqid_a not in result_pairs:
                    result_pairs[sseqid_a] = []

                result_pairs[sseqid_a].append(qseqid_b)
                count += 1
    
# get the local gene list
local_list = []
for sseqid_a, qseqid_b_list in result_pairs.items():
    local_list += qseqid_b_list
            
# Now we need to get the position of genes from the gff file.      

# Read data from the file
df = pd.read_csv(gff_file, sep='\t',comment='#', names=gff_column_names)
# Function to parse the 'Attributes' column and convert it into a dictionary of key-value pairs
def parse_attributes(attribute_str):
    attribute_pairs = attribute_str.split(';')
    attribute_dict = {}
    for pair in attribute_pairs:
        if len(pair.split('=')) == 2:
            key, value = pair.split('=')
        else:
            key = pair.split('=')[0]
            value = ''
        attribute_dict[key] = value
    return attribute_dict
df1 = df['Attributes'].apply(parse_attributes).apply(pd.Series)
file_data = pd.concat([df.drop('Attributes', axis=1), df1], axis=1)

# Filter rows where value from column A is in the list
filtered_rows = file_data[file_data['ID'].isin(local_list)]

# Extract values from column B and C from the filtered rows
result_values = filtered_rows[['Seqid', 'Start','End','ID']]

# Convert columns A and B to numeric
result_values['Start'] = pd.to_numeric(result_values['Start'])
result_values['End'] = pd.to_numeric(result_values['End'])

# Perform the operations and apply the conditions
result_values['Start'] = result_values['Start'] - flanking
result_values['Start'] = result_values['Start'].apply(lambda x: max(0, x))  # Set negative values to 0

result_values['End'] = result_values['End'] + flanking
bedfile = fileA.split('_blastp.tbl')[0] + '_' + str(flanking) + '.bed'
result_values.to_csv(bedfile, sep='\t', index=False)


/opt/anaconda/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/anaconda/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/anaconda/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/

NameError: name 'flankinging' is not defined

In [18]:
df2

,Seqid,Source,Type,Start,End,Score,Strand,Phase,ID,Name,Alias,,Parent,_AED,_QI,_eAED
0,egu.contig.1973,maker,gene,10123,11712,.,+,.,Egu032749,Egu032749,snap_masked-egu.contig.1973-processed-gene-0.0,,NaN,NaN,NaN,NaN
1,egu.contig.1973,maker,mRNA,10123,11712,.,+,.,Egu032749-mRNA-1,Egu032749-mRNA-1,snap_masked-egu.contig.1973-processed-gene-0.0...,,Egu032749,0.06,0|0|0|0.5|1|1|2|0|472,0.04
2,egu.contig.1973,maker,exon,10123,11511,.,+,.,Egu032749-mRNA-1:1,NaN,NaN,,Egu032749-mRNA-1,NaN,NaN,NaN
3,egu.contig.1973,maker,exon,11686,11712,.,+,.,Egu032749-mRNA-1:2,NaN,NaN,,Egu032749-mRNA-1,NaN,NaN,NaN
4,egu.contig.1973,maker,CDS,10123,11511,.,+,0,Egu032749-mRNA-1:cds,NaN,NaN,,Egu032749-mRNA-1,NaN,NaN,NaN
5,egu.contig.1973,maker,CDS,11686,11712,.,+,0,Egu032749-mRNA-1:cds,NaN,NaN,,Egu032749-mRNA-1,NaN,NaN,NaN
6,egu.contig.1477,maker,gene,157297,158409,.,+,.,Egu032743,Egu032743,maker-egu.contig.1477-snap-gene-0.0,,NaN,NaN,NaN,NaN


In [ ]:
df['Attributes'].apply(parse_attributes).apply(pd.Series)

In [10]:
file_data

,Seqid,Source,Type,Start,End,Score,Strand,Phase,Attributes
0,egu.contig.1973,maker,gene,10123,11712,.,+,.,ID=Egu032749;Name=Egu032749;Alias=snap_masked-...
1,egu.contig.1973,maker,mRNA,10123,11712,.,+,.,ID=Egu032749-mRNA-1;Parent=Egu032749;Name=Egu0...
2,egu.contig.1973,maker,exon,10123,11511,.,+,.,ID=Egu032749-mRNA-1:1;Parent=Egu032749-mRNA-1;
3,egu.contig.1973,maker,exon,11686,11712,.,+,.,ID=Egu032749-mRNA-1:2;Parent=Egu032749-mRNA-1;
4,egu.contig.1973,maker,CDS,10123,11511,.,+,0,ID=Egu032749-mRNA-1:cds;Parent=Egu032749-mRNA-1;
5,egu.contig.1973,maker,CDS,11686,11712,.,+,0,ID=Egu032749-mRNA-1:cds;Parent=Egu032749-mRNA-1;
6,egu.contig.1477,maker,gene,157297,158409,.,+,.,ID=Egu032743;Name=Egu032743;Alias=maker-egu.co...
7,egu.contig.1477,maker,mRNA,157297,158409,.,+,.,ID=Egu032743-mRNA-1;Parent=Egu032743;Name=Egu0...
8,egu.contig.1477,maker,exon,157297,157322,.,+,.,ID=Egu032743-mRNA-1:1;Parent=Egu032743-mRNA-1;
9,egu.contig.1477,maker,exon,157819,158409,.,+,.,ID=Egu032743-mRNA-1:2;Parent=Egu032743-mRNA-1;
